In [5]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_excel('D:\\data files\\Copy of emails.xlsx')
print("file loaded")

file loaded


In [8]:
print(df.shape)
df.head()

(5172, 3002)


,Email No.,the,to,ect,and,for,of,a,you,hou,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
0,Email 1,0,0,1,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Email 2,8,13,24,6,6,2,102,1,27,...,0,0,0,0,0,0,0,1,0,0
2,Email 3,0,0,1,0,0,0,8,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Email 4,0,5,22,0,5,1,51,2,10,...,0,0,0,0,0,0,0,0,0,0
4,Email 5,7,6,17,1,5,2,57,0,9,...,0,0,0,0,0,0,0,1,0,0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5172 entries, 0 to 5171
Columns: 3002 entries, Email No. to Prediction
dtypes: int64(3001), object(1)
memory usage: 118.5+ MB


In [13]:
print(df.columns[:20])

Index(['Email No.', 'the', 'to', 'ect', 'and', 'for', 'of', 'a', 'you', 'hou',
       'in', 'on', 'is', 'this', 'enron', 'i', 'be', 'that', 'will', 'have'],
      dtype='object')


In [15]:
df = df.loc[:, ~df.columns.astype(str).str.contains("^Unnamed", na = False)]
df = df.drop(columns = ['Email No.'])
print(df.columns)

Index([           'the',             'to',            'ect',            'and',
                  'for',             'of',              'a',            'you',
                  'hou',             'in',
       ...
             'connevey',            'jay',         'valued',            'lay',
       'infrastructure',       'military',       'allowing',             'ff',
                  'dry',     'Prediction'],
      dtype='object', length=3001)


In [17]:
df = df.loc[:, df.nunique()>1]
print(df.columns)

Index([           'the',             'to',            'ect',            'and',
                  'for',             'of',              'a',            'you',
                  'hou',             'in',
       ...
             'connevey',            'jay',         'valued',            'lay',
       'infrastructure',       'military',       'allowing',             'ff',
                  'dry',     'Prediction'],
      dtype='object', length=3001)


In [19]:
X = df.drop(columns = ['Prediction'])
Y = df['Prediction']
X.columns = X.columns.astype(str)

In [21]:
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
X_scaled = scalar.fit_transform(X)

In [25]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.95)
X_pca = pca.fit_transform(X_scaled)

print("Original shape: ", X.shape)
print("PCA shape: ", X_pca.shape)

Original shape:  (5172, 3000)
PCA shape:  (5172, 1063)


In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_pca, Y, test_size = 0.2, random_state = 42)

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       739
           1       0.97      0.87      0.92       296

    accuracy                           0.96      1035
   macro avg       0.96      0.93      0.94      1035
weighted avg       0.96      0.96      0.95      1035



In [31]:
y_prob = model.predict_proba(X_test)[:, 1]
threshold = 0.4
y_pred_adjusted = (y_prob >= threshold).astype(int)
print(classification_report(y_test, y_pred_adjusted))

              precision    recall  f1-score   support

           0       0.99      0.96      0.98       739
           1       0.92      0.99      0.95       296

    accuracy                           0.97      1035
   macro avg       0.96      0.98      0.97      1035
weighted avg       0.97      0.97      0.97      1035



In [33]:
import os
print(os.getcwd())


C:\Users\Aries


In [53]:
import joblib
joblib.dump(model, "spam_model.pkl")

['spam_model.pkl']

In [55]:
joblib.dump(scalar, "scalar.pkl")

['scalar.pkl']

In [57]:
joblib.dump(pca, "pca.pkl")

['pca.pkl']

In [59]:
from sklearn.pipeline import Pipeline
scaler = joblib.load("scalar.pkl")
pca = joblib.load("pca.pkl")
model = joblib.load("spam_model.pkl")

pipeline = Pipeline([
    ('scaler', scaler),
    ('pca', pca),
    ('model', model)
])

joblib.dump(pipeline, 'spam_pipeline.pkl')

['spam_pipeline.pkl']